# Mapping geographical zones between model and GCMR

This notebook finds fuzzy matches for countries and regions between
file `model_zones_file` and the "Google Community Mobility Reports" (GCMR) data.

In [1]:
import pandas as pd
import urllib.request
import os
import numpy as np
import pylcs

## Loading GCMR data

In [2]:
GCMR_file_path = "./Global_Mobility_Report.csv"

Removes previous file

In [3]:
if os.path.isfile(GCMR_file_path):
    os.remove(GCMR_file_path)
else:
    print ("File not exists")

Downloads the latest GCMR data

In [4]:
print('Start the download')

try :
    url = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=e0c5a582159f5662'
    urllib.request.urlretrieve(url, GCMR_file_path)
    print('The file have been correctly downloaded')
except Exception as e:
    print("The file have NOT been correctly downloaded")
    print(e)

Start the download
The file have been correctly downloaded


In [5]:
gcmr_df = pd.read_csv("./Global_Mobility_Report.csv")

## Loading covid 19 model countries

The user will have prepared a csv file with two columns `covid19model_country` and `covid19model_region`. Optionnaly `GCMR_country` and `GCMR_region` can be defined and left empty, or partially filled, the rest of the program uses fuzzy search to find them

In [6]:
model_zones_file = 'covid19model_zones.csv'
model_zones_df = pd.read_csv(model_zones_file)

In [7]:
gcmr_df.sample(10)

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
159106,US,United States,Louisiana,Natchitoches Parish,2020-03-15,8.0,26.0,NaN,NaN,0.0,NaN
143652,US,United States,Iowa,Page County,2020-04-21,NaN,NaN,NaN,NaN,-26.0,NaN
170608,US,United States,Michigan,Van Buren County,2020-03-29,-43.0,-24.0,NaN,NaN,-30.0,NaN
198686,US,United States,New Mexico,McKinley County,2020-03-31,-33.0,8.0,NaN,-11.0,-47.0,19.0
167707,US,United States,Michigan,Iron County,2020-02-23,23.0,NaN,NaN,NaN,NaN,NaN
113542,US,United States,Georgia,Bryan County,2020-04-22,-30.0,-11.0,NaN,-8.0,-44.0,18.0
156337,US,United States,Kentucky,Warren County,2020-04-11,-45.0,-13.0,-43.0,NaN,-37.0,12.0
32301,GB,United Kingdom,Midlothian,NaN,2020-04-09,-77.0,-24.0,NaN,-54.0,-72.0,NaN
107316,US,United States,Delaware,NaN,2020-03-11,7.0,7.0,24.0,1.0,1.0,0.0
258237,US,United States,Utah,Davis County,2020-04-10,-38.0,7.0,53.0,-30.0,-54.0,21.0


In [8]:
df_obj = model_zones_df.select_dtypes(['object'])
model_zones_df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip().str.strip('"'))
model_zones_df.sample(4)

,covid19model_country,covid19model_region,GCMR_country,GCMR_region
26,Switzerland,Switzerland,,
11,France,Provence-Alpes-Côte d'Azur,,
2,France,Bretagne,,
18,Germany,Germany,,


In [9]:
for GCMR_field in ["GCMR_country",	"GCMR_region"]:
    try:
        model_zones_df[GCMR_field]
    except KeyError as e:
        model_zones_df[GCMR_field] = np.nan
    field_na_indices = (
        (model_zones_df[GCMR_field] == ' ') 
        | (model_zones_df[GCMR_field] == '') 
        | (model_zones_df[GCMR_field].isna())
    )
    model_zones_df.loc[field_na_indices, GCMR_field] = None

model_zones_df.sample(4)

,covid19model_country,covid19model_region,GCMR_country,GCMR_region
21,France,France,None,None
8,France,Nouvelle-Aquitaine,None,None
3,France,Centre-Val de Loire,None,None
19,Spain,Spain,None,None


## Matching model geographical zones to a zone of the GCMR

In [10]:
gcmr_countries = gcmr_df["country_region"].unique()
print(f"{len(gcmr_countries)} countries in the GCMR report")

132 countries in the GCMR report


Use fuzzy logic to match strings with a metric including:

- Longest common subsequence (added)
- Longest common substring (added)
- Non-matched characters (substracted)
- Divided by the length of the original string.

In [11]:
def  most_probable_match(string, possible_strings, n_matches=1):
    """ Fuzzy lookup for a similar string
    """

    def return_match(arg_most_prob, probable_match, i_match=1):
        print(
            f"\tMatch {i_match} of {n_matches} : "
            + f"'{possible_strings[arg_most_prob]}' == '{string}'  "
            + f"with a match score of {probable_match[arg_most_prob]}/{2*len(string)}."
        )
        return possible_strings[arg_most_prob]

    lcsubsequence_list = pylcs.lcs_of_list(string, possible_strings)
    probable_match = (np.add(
        lcsubsequence_list,  # longest subsequence 
        pylcs.lcs2_of_list(string, possible_strings) # longest substring
    ).astype(np.int) - (
        np.array([len(t) - m for t, m in zip(possible_strings, lcsubsequence_list)])  # Penalise extra characters in the match
    ))
    if n_matches == 1:
        arg_most_prob = np.argmax(probable_match)
        return return_match(arg_most_prob, probable_match)
    elif n_matches > 1:
        args_most_prob = np.argpartition(probable_match, -n_matches)[-1:-n_matches-1:-1]
        return [return_match(a, probable_match, i+1) for i, a in enumerate(args_most_prob)]
    else: 
        raise AttributeError("Invalid number of matches requested")

elow are some examples of the scores, differences in length are heavily penalised.

In [12]:
print("Testing single returns:")
most_probable_match("Normandie", ["Paris"])
most_probable_match("Normandie", ["France", "Paris"])
most_probable_match("Normandie", ["France", "Paris", "Normandie Bretagne"])
most_probable_match("Normandie", ["Normandy", "France", "Paris", "Normandie Bretagne"])

print("Testing multiple value return:")
most_probable_match("Normandie", 
                    ["Normandy", "France", "Paris", 
                     "Normandie Bretagne", "Normandie"
                    ],
                    n_matches=3
)


Testing single returns:
	Match 1 of 1 : 'Paris' == 'Normandie'  with a match score of 0/18.
	Match 1 of 1 : 'France' == 'Normandie'  with a match score of 4/18.
	Match 1 of 1 : 'Normandie Bretagne' == 'Normandie'  with a match score of 9/18.
	Match 1 of 1 : 'Normandy' == 'Normandie'  with a match score of 13/18.
Testing multiple value return:
	Match 1 of 3 : 'Normandie' == 'Normandie'  with a match score of 18/18.
	Match 2 of 3 : 'Normandy' == 'Normandie'  with a match score of 13/18.
	Match 3 of 3 : 'Normandie Bretagne' == 'Normandie'  with a match score of 9/18.


['Normandie', 'Normandy', 'Normandie Bretagne']

### Matching countries

In [13]:
model_to_GCMR_countries = { 
    c: c for c in model_zones_df["covid19model_country"].unique()
} 

for country in model_to_GCMR_countries:
    if country not in gcmr_countries:
        print(f"WARNING: an exact match for country '{country}' "
              + "could not be found in the GCMR database")
        model_to_GCMR_countries[country] = most_probable_match(country, gcmr_countries)

# Fill in the "GCMR_country" column
for index, row in model_zones_df.iterrows():
    if row["GCMR_country"] is None or row["GCMR_country"] not in gcmr_countries:
        model_zones_df.loc[index ,"GCMR_country"] = model_to_GCMR_countries[row["covid19model_country"]]

	Match 1 of 1 : 'United Kingdom' == 'United_Kingdom'  with a match score of 19/28.


Check that the dataframe is populated

In [14]:
print("Sample below should show 'GCMR_country' column populated")
# Country column is prepared
model_zones_df.sample(4)

Sample below should show 'GCMR_country' column populated


,covid19model_country,covid19model_region,GCMR_country,GCMR_region
9,France,Occitanie,France,None
29,Netherlands,Netherlands,Netherlands,None
12,France,Île-de-France,France,None
18,Germany,Germany,Germany,None


### Matching regions

Regions are matched to the closest match in the parent country using `most_probable_match`. YOu may need to customise the function for countries which have different alphabets.

In [15]:
# Preparing the region columns in the same way
for index, row in model_zones_df.iterrows():
    region = row["covid19model_region"]
    country = row["GCMR_country"]
    # Extract for the country the list of regions
    gcmr_regions = gcmr_df[gcmr_df["country_region"] == country]["sub_region_1"]
    gcmr_regions[gcmr_regions.isna()] = country
    gcmr_regions = gcmr_regions.unique()
    # if the region and country have exactly the same name use the country as the zone
    if region == row["covid19model_country"]:
        model_zones_df.loc[index, "GCMR_region"] = row["GCMR_country"]
    
    # Otherwise if it exists as is in GCMR copy it over
    elif region in gcmr_regions:
        model_zones_df.loc[index, "GCMR_region"] = region
    # Else use fuzzy logic to get a best match
    else:
        print(f"WARNING: an exact match for region '{region}' in {country} could not be found in the GCMR database")
        model_zones_df.loc[index, "GCMR_region"] = most_probable_match(region, gcmr_regions)

	Match 1 of 1 : 'Brittany' == 'Bretagne'  with a match score of 4/16.
	Match 1 of 1 : 'Corsica' == 'Corse'  with a match score of 5/10.
	Match 1 of 1 : 'Normandy' == 'Normandie'  with a match score of 13/18.
	Match 1 of 1 : 'France' == 'France-hopitaux'  with a match score of 12/30.
	Match 1 of 1 : 'France' == 'France-OC19'  with a match score of 12/22.
	Match 1 of 1 : 'France' == 'France-EHPAD'  with a match score of 12/24.


Now we can check that data frame is correctly populated:

In [16]:
print("Sample below should show 'GCMR_region' column populated")
model_zones_df.sample(4)

Sample below should show 'GCMR_region' column populated


,covid19model_country,covid19model_region,GCMR_country,GCMR_region
11,France,Provence-Alpes-Côte d'Azur,France,Provence-Alpes-Côte d'Azur
4,France,Corse,France,Corsica
9,France,Occitanie,France,Occitanie
5,France,Grand Est,France,Grand Est


## Ouptut to CSV

The resulting data frame is written to csv ready to be used in other parts of the program

In [17]:
filename, ext= os.path.splitext(model_zones_file)
prepared_model_zones_file = filename + '_prepared' + ext
i = 1
while os.path.exists(prepared_model_zones_file):
    prepared_model_zones_file = filename + '_prepared' + f"_{i}" + ext
    i += 1

model_zones_df.to_csv(prepared_model_zones_file)

print(
    "Prepared country and region matches between model and GCMR data has been " 
    + f"printed to {prepared_model_zones_file}, please check warnings"
    + " above, and edit the csv file directly if needed."
)

Prepared country and region matches between model and GCMR data has been printed to covid19model_zones_prepared.csv, please check warnings above, and edit the csv file directly if needed.
